# **Resolución TP1: Algoritmos de búsqueda en Torre de Hanoi**

## 1. ¿Cuáles son los PEAS de este problema?

###

- **Perfomance**: partiendo del estado inicial (cinco discos apilados de en varilla 1) moverlos a varilla 3, respetando las reglas del juego.

- **Enviroment**: cinco discos de distintos tamaños, base con tres varillas.

- **Actuators**:

- **Sensors**: detección del estado actual de las varillas (discos en cada una y orden de esos discos).

## 2. ¿Cuáles son las propiedades del entorno de trabajo?

###

- **Totalmente observable**: el agente puede ver en todo momento el estado del entorno (posición de los discos en cada varilla).

- **Determinista**: el siguiente estado esta determinado por el estado actual y la acción que ejecuta el agente.

- **Secuencial**: la decisión actual puede afectar las decisiones futuras.

- **Estático**: el entorno no se modifica mientras el agente delibera.

- **Discreto**: tiene un número finito de estados distintos.

- **Agente individual**
